

# SQLite examples



![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)


In [2]:
import sqlite3 as sqlite3
from creating_table import creating_table 

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)
## SQL Movie-Rating Query Exercises Extras
### Tables:
- 1. Movie ( mID, title, year, director ) 
- 2. Reviewer ( rID, name ).
- 3. Rating ( rID, mID, stars, ratingDate ).

### 1.Creating tables:

In [4]:
connection = sqlite3.connect(":memory:")
cursor = connection.cursor()
creating_table("Movie.txt", cursor, connection)
creating_table("Reviewer.txt", cursor, connection)
creating_table("Rating.txt", cursor, connection)
rows = cursor.execute("SELECT * FROM Movie;", "").fetchone()
print(rows)
rows = cursor.execute("SELECT * FROM Reviewer;", "").fetchone()
print(rows)
rows = cursor.execute("SELECT * FROM Rating;", "").fetchone()
print(rows)

(101, 'Gone with the Wind', 1939, 'Victor Fleming')
(201, 'Sarah Martinez')
(201, 101, 2, '2011-01-22')


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## 2.  Examples:

- Find the names of all reviewers who rated Gone with the Wind.

In [5]:
sql_str = '''Select distinct name from Rating join Reviewer on Reviewer.rID = Rating.rID 
            where Rating.mID in (Select mID from Movie where title like '%Gone with the Wind%');'''
rows = cursor.execute(sql_str).fetchone()
print(rows)

('Sarah Martinez',)


- For any rating where the reviewer is the same as the director of the movie, return the reviewer name, movie title, and number of stars.

In [6]:
sql_str = ''' Select distinct name, title, stars from Rating join Reviewer on Reviewer.rID = Rating.rID  
            Join Movie on Rating.mID = Movie.mID and Movie.director = Reviewer.name;'''
rows = cursor.execute(sql_str).fetchone()
print(rows)

('James Cameron', 'Avatar', 5)


- Return all reviewer names and movie names together in a single list, alphabetized. (Sorting by the first name of the reviewer and first word in the title is fine; no need for special processing on last names or removing "The".)  

In [7]:
sql_str = ''' Select distinct title NT  from  Movie union Select name from Reviewer order by NT;'''
rows = cursor.execute(sql_str).fetchone()
print(rows)

('Ashley White',)


- Find the titles of all movies not reviewed by Chris Jackson. 

In [8]:
sql_str = ''' Select title from Movie where mID in (select mID from Rating  
            where mID not in (Select mID from Rating  where rID in (Select rID 
            from Reviewer where name like '%Chris Jackson%' ))) or mID not in(select mID from Rating  );'''
rows = cursor.execute(sql_str).fetchone()
print(rows)

('Gone with the Wind',)


- For all pairs of reviewers such that both reviewers gave a rating to the same movie, return the names of both reviewers. Eliminate duplicates, don't pair reviewers with themselves, and include each pair only once. For each pair, return the names in the pair in alphabetical order.

In [9]:
sql_str = ''' Select distinct  N1, N2 from (Select distinct  CASE WHEN Reviewer.name > Reviewer1.name THEN Reviewer1.name  
            ELSE Reviewer.name END N1, CASE WHEN Reviewer.name > Reviewer1.name THEN Reviewer.name ELSE Reviewer1.name END N2 
            From (Select rID rID1, mID mID1 from Rating group by rID1, mID1) left   join (Select rID rID2, mID mID2  from Rating  group by rID2, mID2) 
            on mID1 = mID2 join Reviewer on rID1 = Reviewer.rID  join Reviewer as Reviewer1 on rID2 = Reviewer1.rID  where rID1<>rID2) order by N1;'''
rows = cursor.execute(sql_str).fetchone()
print(rows)

('Ashley White', 'Chris Jackson')


- For each rating that is the lowest (fewest stars) currently in the database, return the reviewer name, movie title, and number of stars. 

In [10]:
sql_str = ''' Select distinct name, title, stars from Rating  
            join Movie on Rating.mID = Movie.mID join Reviewer  on Rating.rID = Reviewer.rID where stars in (select min(stars) from Rating);'''
rows = cursor.execute(sql_str).fetchone()
print(rows)

('Sarah Martinez', 'Gone with the Wind', 2)


- List movie titles and average ratings, from highest-rated to lowest-rated. If two or more movies have the same average rating, list them in alphabetical order. 

In [12]:
sql_str = ''' Select  title, astr from (Select avg(stars) astr, mID mID1 from Rating group by mID1)   
            join Movie on Movie.mID = mID1 order by astr desc, title ;'''
rows = cursor.execute(sql_str).fetchone()
print(rows)

('Snow White', 4.5)


- Find the names of all reviewers who have contributed three or more ratings. (As an extra challenge, try writing the query without HAVING or without COUNT.) 

In [13]:
sql_str = ''' Select name  from (Select rID rID1,  sum(c1)  c2 from (Select rID, 1 c1 from Rating) group by rID) 
            join Reviewer on Reviewer.rID = rID1 where c2 >=3;'''
rows = cursor.execute(sql_str).fetchone()
print(rows)

('Brittany Harris',)


- Some directors directed more than one movie. For all such directors, return the titles of all movies directed by them, along with the director name. Sort by director name, then movie title. (As an extra challenge, try writing the query both with and without COUNT.) 

In [14]:
sql_str = ''' Select title, director from Movie join (select director  D1,  sum(c1)  c2 from (Select director, 1 c1 from Movie) group by director ) 
            on Movie.director = D1 where c2 >=2 order by director, title;'''
rows = cursor.execute(sql_str).fetchone()
print(rows)

('Avatar', 'James Cameron')


- Find the movie(s) with the highest average rating. Return the movie title(s) and average rating. (Hint: This query is more difficult to write in SQLite than other systems; you might think of it as finding the highest average rating and then choosing the movie(s) with that average rating.) 

In [15]:
sql_str = ''' Select title, av_stars  from (Select mID mID1, avg(stars) av_stars from Rating group by mID) join Movie on mID1 = Movie.mID  and  av_stars  in  (select max(av_stars ) 
            from (Select mID mID1, avg(stars) av_stars from Rating group by mID));'''
rows = cursor.execute(sql_str).fetchone()
print(rows)

('Snow White', 4.5)


- Find the movie(s) with the lowest average rating. Return the movie title(s) and average rating. (Hint: This query may be more difficult to write in SQLite than other systems; you might think of it as finding the lowest average rating and then choosing the movie(s) with that average rating.) 

In [17]:
sql_str = ''' Select title, av_stars  from (Select mID mID1, avg(stars) av_stars from Rating group by mID) 
            join Movie on mID1 = Movie.mID  and  av_stars  in  (select min(av_stars ) 
            from (Select mID mID1, avg(stars) av_stars from Rating group by mID));'''
rows = cursor.execute(sql_str).fetchone()
print(rows)

('The Sound of Music', 2.5)


- For each director, return the director's name together with the title(s) of the movie(s) they directed that received the highest rating among all of their movies, and the value of that rating. Ignore movies whose director is NULL. 

In [19]:
sql_str = ''' Select distinct director_, title, m_satrs from Movie join (Select distinct  director director_, max(stars)  m_satrs, Movie.mID MmID from Rating 
            join Movie on Movie.mID = Rating.mID group by director_) 
            on Movie.mID = MmID  where  director is not NULL order by  director_;'''
rows = cursor.execute(sql_str).fetchone()
print(rows)

('James Cameron', 'Avatar', 5)


![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)